 ### html파일 생성

In [ ]:
#!pip install pyhwp or conda install pyhwp
# !hwp5html 변경할 hwp경로    ---> hwp파일과 같은 이름의 폴더 안에 html 생성
!hwp5html ./Daejeon.hwp 

### html파일 파싱 예시

In [4]:
from urllib import request
from bs4 import BeautifulSoup
import html5lib
import pandas as pd
import os
import re
path = os.getcwd().replace('\\','/')

# 저장된 xhtml를 호출
url_1 = "/Daejeon/index.xhtml"
url = 'file:///'+path + url_1

with request.urlopen(url) as f:
    html = f.read().decode('utf-8')
bs = BeautifulSoup(html, 'html.parser')

In [2]:
# p 18 ~ p 23 까지 목차
# . : class , # : id
# 띄어쓰기를 꼭 해야함
a = bs.select('body > div .Page > p')  
#a = bs.select('body > .Section.Section-0.Paper > div > .Page')

In [ ]:
# 처음 목차가 시작하는 지점을 찾음
p = re.compile('목[ ]*차|차[ ]*례|순[ ]*서')
s= 0
for i in a:
    if  len(p.findall(i.text)) != 0:
        print(s)
        break
    s += 1

In [6]:
for i in a[s]:
    for j in i:
        try:
            print(j.text)
        except: pass

목   차 
제1장 운영현황 
 제1절 운영현황    3 
   1.1.1.  119종합상황실 설치규정    3 
   1.1.2.  119종합상황실 운영규정 및 연혁    3 
   1.1.3.  119종합상황실 주요업무    4 
   1.1.4.  근무현황    4 
제2장 운영시스템 
 제1절 긴급구조표준시스템    7 
   2.1.1.  지령운영(신고접수/출동지령/지령접수)    7 
   2.1.2.  관제운영(상환관제, 상황운영지원)    8 
   2.1.3.  정보지원시스템    9 
   2.1.4.  GIS 및 공동시스템    10 
   2.1.5.  통계시스템   11 
   2.1.6.  녹취시스템   12 
   2.1.7.  무선통신제어시스템   13 
   2.1.8.  현장영상전송시스템   14 
   2.1.9.  시스템 및 운영 안정성 확보   15 
   2.1.10. 예비상황실 운영   16 
 제2절 기타 운영시스템  17 
   2.2.1.  119안전신고센터   17 
   2.2.2.  이동전화 위치정보 조회시스템   18 
   2.2.3.  화학물질 사고대응정보시스템   19 
   2.2.4.  지하공동구 관리시스템   20 
   2.2.5.  긴급신고전화 통합시스템   21 
   2.2.6.  외국인 119신고접수 처리 운용   22 


### 카테고리 추출

* html의 본문을 파싱할 시 사용하는 목차 번호 및 이름을 저장


In [ ]:
# 세부카테고리 추출
# 범위지정이 아니라 범용적으로 추출할 수 있도록 변경이 필요함

# 전체 목차 리스트
idx_list_all  = [a[i].select('p')[j].text for i in range(18,24) for j in range(len(a[i].select('p'))) ]

# 제 1장 형식을 찾는 정규식
p = re.compile('제[0-9]장')

# 1.1.1 형식을 찾는 정규식
p1= re.compile('[0-9]{1,2}.[0-9]{1,2}.[0-9]{1,2}')

In [8]:
# 장과 내부 데이터를 합쳐서 추출
ex_list = []
for i in idx_list_all:
    # 제1장 운영현황 등의 대목차를 추출
    if p.search(i):
        tmp_c = i.strip().split(' ',1)
        
    # 1.1.1. 119종합상황실 설치규정 등 목차를 추츨
    elif i.find('.') != -1:
        # 6.3.1 오타있음 유해화학물질 유출사고 시 상'환'관리요원 표준행동절차 -> 유해화학물질 유출사고 시 상'황'관리요원 표준행동절차
        # 6.3.1 뒤에 . 하나가 누락
        # 목차 중 한글만 추출
        tmp_v = i[i.rfind('.')+2:-6].strip()  
        
        # 정규식을 통해 1.1.1 형식의 숫자를 추출
        tmp_n = p1.findall(i)[0]
        
        ex_list.append([tmp_c[0],tmp_c[1],tmp_n, tmp_v])
        # 6번데 오타있음
# input으로 사용할 데이터 값들
full_df = pd.DataFrame(ex_list)
full_df[4] = full_df[0]+' '+full_df[2]+' '+full_df[3]
 

In [9]:
# 본문을 호출할 index.csv 파일, 추후에 화제유형을 임의로 지정하여 호출
full_df  

,0,1,2,3,4
0,제1장,운영현황,1.1.1,119종합상황실 설치규정,제1장 1.1.1 119종합상황실 설치규정
1,제1장,운영현황,1.1.2,119종합상황실 운영규정 및 연혁,제1장 1.1.2 119종합상황실 운영규정 및 연혁
2,제1장,운영현황,1.1.3,119종합상황실 주요업무,제1장 1.1.3 119종합상황실 주요업무
3,제1장,운영현황,1.1.4,근무현황,제1장 1.1.4 근무현황
4,제2장,운영시스템,2.1.1,지령운영(신고접수/출동지령/지령접수),제2장 2.1.1 지령운영(신고접수/출동지령/지령접수)
...,...,...,...,...,...
129,제9장,대민지원활동 신고접수,11.4.3,민원처리 결과 입력,제9장 11.4.3 민원처리 결과 입력
130,제9장,대민지원활동 신고접수,11.4.4,신고일원화 시스템 현황 정비,제9장 11.4.4 신고일원화 시스템 현황 정비
131,제9장,대민지원활동 신고접수,12.1.1,상황보고체계개요,제9장 12.1.1 상황보고체계개요
132,제9장,대민지원활동 신고접수,12.1.2,상황보고체계기준,제9장 12.1.2 상황보고체계기준


In [ ]:
 
full_df.to_csv('./data/index.csv', index = False )